**Run this script on every Sunday.**

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.arzt-auskunft.de/hamburg/"

In [3]:
def checkResponse(response):
  try:
    response.raise_for_status()  # Prüfen, ob die Anfrage erfolgreich war
  except requests.exceptions.HTTPError as err:
    print(err)

  return response.status_code

In [4]:
def scrapeOnePage(medicalOfficeList, response):
  soup = BeautifulSoup(response.text, 'html.parser')
  arztpraxen = soup.find_all('div', class_='card dl mb-3')

  for praxis in arztpraxen:
    praxisname = praxis.find('h3', class_='card-title').text.strip()
    if praxisname != "" and praxisname not in medicalOfficeList:
      medicalOfficeList.append(praxisname)
      #print(f"{praxisname}\n")

In [5]:
zähler = 0
arztPraxenListe = []

In [6]:
for i in range(200):
  if zähler <= 1:
    url = "https://www.arzt-auskunft.de/hamburg/"
  else:
    url = "https://www.arzt-auskunft.de/hamburg/" + str(zähler) + "/"

  response = requests.get(url)
  statusCode = checkResponse(response)

  if statusCode != 404:
    scrapeOnePage(arztPraxenListe, response)
    zähler += 1
  else:
    print("Ende")
    break


404 Client Error: Not Found for url: https://www.arzt-auskunft.de/hamburg/119/
Ende


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# prompt: export the list variable arztpraxen_as_list to an excel file with today's date as the file name

import pandas as pd
from datetime import date, timedelta

# Get today's date
today = date.today().strftime("%Y-%m-%d")

# Create a DataFrame from the list
todayDF = pd.DataFrame(arztPraxenListe, columns=['Praxisname'])

# Export the DataFrame to an Excel file with today's date as the file name
todayDF.to_excel(f"{today}.xlsx", index=False)

#from google.colab import files
#files.download(f"{today}.xlsx")

In [10]:
lastCheckpoint = date.today() - timedelta(days=7)
lastCheckpointDF = pd.read_excel(f"{today}.xlsx")
#lastCheckpointDF = pd.read_excel(f"{lastCheckpoint.strftime('%Y-%m-%d')}.xlsx")

df_diff = pd.concat([lastCheckpointDF, todayDF]).drop_duplicates(keep=False)

if len(df_diff.index) != 0:
  df_diff.to_excel(f"{today}-{lastCheckpoint.strftime('%Y-%m-%d')}.xlsx", index=False)